In [13]:
import os
os.chdir("..")

In [14]:
import django
from django.conf import settings
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'mimuse.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [57]:
import json
import http.client

from songs.models import Album, Song
from artists.models import Artist

In [16]:
conn = http.client.HTTPSConnection("spotify23.p.rapidapi.com")

headers = {
    'X-RapidAPI-Key': "6e75800c20msh1e3b52b20821a7dp1194edjsn5afc9a2de8d4",
    'X-RapidAPI-Host': "spotify23.p.rapidapi.com"
}

conn.request("GET", "/artist_albums/?id=2w9zwq3AktTeYYMuhMjju8&offset=0&limit=100", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"data":{"artist":{"discography":{"albums":{"totalCount":11,"items":[{"releases":{"items":[{"id":"5aWF6mKr3bSRlMnyxA8Tip","uri":"spotify:album:5aWF6mKr3bSRlMnyxA8Tip","name":"Champagne Problems #DQH2","type":"ALBUM","date":{"year":2022,"isoString":"2022-03-11T00:00:00Z"},"coverArt":{"sources":[{"url":"https:\/\/i.scdn.co\/image\/ab67616d00001e02fec9341c25339773dfe59cb5","width":300,"height":300},{"url":"https:\/\/i.scdn.co\/image\/ab67616d00004851fec9341c25339773dfe59cb5","width":64,"height":64},{"url":"https:\/\/i.scdn.co\/image\/ab67616d0000b273fec9341c25339773dfe59cb5","width":640,"height":640}]},"playability":{"playable":true,"reason":"PLAYABLE"},"sharingInfo":{"shareId":"S2nULwTAQkC9LdppmGc0aw","shareUrl":"https:\/\/open.spotify.com\/album\/5aWF6mKr3bSRlMnyxA8Tip?si=S2nULwTAQkC9LdppmGc0aw"},"tracks":{"totalCount":8}}]}},{"releases":{"items":[{"id":"62XwEYC2iJIX37oZlJbsvi","uri":"spotify:album:62XwEYC2iJIX37oZlJbsvi","name":"Champagne Problems #DQH1","type":"ALBUM","date":{"year":2

In [17]:
import html
import bs4

In [ ]:
conn = http.client.HTTPSConnection("spotify23.p.rapidapi.com")

headers = {
    'X-RapidAPI-Key': "6e75800c20msh1e3b52b20821a7dp1194edjsn5afc9a2de8d4",
    'X-RapidAPI-Host': "spotify23.p.rapidapi.com"
}


data = Artist.objects.all()

for i in data:
    artist_id = str(i)
    print(i)
    conn.request("GET", "/artist_albums/?id={artist_id}".format(artist_id=artist_id), headers=headers)
    res = conn.getresponse()
    datas = json.loads(res.read().decode('utf-8'))
    print(datas)
    num_albums = datas['data']['artist']['discography']['albums']['totalCount']
    x = 0
    for album in datas['data']['artist']['discography']['albums']['items']:
        this = Album(api_id = album['releases']['items'][0]['id'])
        this.save()
        artist = Artist.objects.get(api_id=artist_id)
        artist.albums.add(this)
        artist.save()

In [ ]:
conn = http.client.HTTPSConnection("spotify23.p.rapidapi.com")

headers = {
    'X-RapidAPI-Key': "d2fe19cdebmshffcb0a0ee9a9c36p16e57cjsn0393ba7168eb",
    'X-RapidAPI-Host': "spotify23.p.rapidapi.com"
}

for artist in Artist.objects.all():
    albums = Artist.albums.through.objects.filter(artist_id=artist.id)
    for album in albums:
        album_id = Album.objects.get(id=album.album_id).api_id
        artist_id = Artist.objects.get(id=album.artist_id).api_id
        print(album_id, artist_id) 
        conn.request("GET", "/albums/?ids={album_id}".format(album_id=album_id), headers=headers)
        res = conn.getresponse()
        album_info = json.loads(res.read().decode('utf-8'))['albums'][0]
        print(album_info)
        this_album = Album.objects.filter(api_id=album_id)
        print(this_album)
        if(album_info['release_date_precision']=="day"):
            this_album.update(title=album_info['name'], release_date=album_info['release_date'], cover_image=album_info['images'][0]['url'])
        else:
            this_album.update(title=album_info['name'], cover_image=album_info['images'][0]['url'])
        this_artist = Artist.objects.get(api_id=artist_id)
        this_artist.albums.add(this_album.first())    
        this_artist.save()

In [71]:
conn = http.client.HTTPSConnection("spotify23.p.rapidapi.com")

headers = {
    'X-RapidAPI-Key': "492a68091cmsh28fa0cbbfc77d53p1a5ad0jsnc64f45e3f32c",
    'X-RapidAPI-Host': "spotify23.p.rapidapi.com"
}

In [72]:
for album in Album.objects.all():
    album_id = album.api_id
    conn.request("GET", "/album_tracks/?id={album_id}&limit=20".format(album_id=album_id), headers=headers)
    res = conn.getresponse()
    album_tracks = json.loads(res.read().decode('utf-8'))['data']['album']['tracks']['items']
    for track in album_tracks:
        uri = track['track']['uri']
        id = uri.replace('spotify:track:', '')
        print(id)
        conn.request("GET", "/tracks/?ids={id}".format(id=id), headers=headers)
        res = conn.getresponse()
        track_info = json.loads(res.read().decode('utf-8'))['tracks'][0]
        if Song.objects.filter(api_id=id).exists():
            continue
        else:
            try:
                Song.objects.update_or_create(api_id=id, album_id=album, title=track_info['name'], popularity=track_info['popularity'], preview_url = track_info['preview_url'], release_date=album.release_date)
            except:
                print("TOO MANY SONGS API CANNOT HANDLE")

3ym8svDx03YE9WDdLOOg1z
1nSC2yK3EFDX11iwlXuaDm
0lgjN8m6N7SndvLmwo5nm7
79qKDXwVdpsalZvRX8eq0k
5ecmJO7p9d4OSTUq3JiH7A
751VCyYR0ZLHbz2IzBXUPy
5I7J0XnpZBQ3nrTLsfEMMX
6yNbgJy8Gh9PwQpYH7Vw8q
3iA3IOaEFiwsKHbMa2FzIH
2ZHtttFviE7UPYsVyKyEeH
2QKCvP63SA1f2Qyuu3pRuo
32Ron8UKy1My0dA5NG0gv5
5bhilpY0QN9YCcQ9hlheeI
5JmeqPQVJcR9zclh8Zz2TM
4awhKDwkMoK18NUIR4aBkM
1hzM6iNe6D87LP0dQqALoL
0cRkE0KU3YVrHyosYPm7gh
0F2kMWTJsbakPYDmJ8bcxv
5vFBANNZp78umWqKFLQXo5
3iEur7dFsYi2ZrNhvlLcab
6QaUkb9oJ2zoSQL3WfQxzJ
6iYb7wTWlptHfD0hwdCHsn
31SXbc8jA2y1Q4xfMAASA8
4l1zFB1UoU9LQh5TNlYKiH
1x89HkJY0M6Lu9XYdtdsjG
2oZVeK4zybKD8TZW1bdsY4
1B1aFbbk02C6tbVD7LzvYK
4c8fJhZ8EzT3mYnwfvT42C
2ENZ9lWpoLAGFDJsDb2Cbt
6viJhP1omFP37DdRHO6Gk7
1PoqPHvqIIZY9QOCFXWjre
7LvZFvfo7Vr5i84GZ44JGr
0Dgto5V6NRLTqWaGKOUAtd
2HpYrrrf4mA84To3PCsvSR
30qmq9UtzeM9QVy4Csn3V3
1IdM9Qtv8BE9IZ3BNiPeMA
6YnGrwsjBSk3oGo4aJdo2Q
2UO5SzVbSWubanngPvePXu
5VOxuSoBdFdjzbLsUT2VsX
4neyZ9fJFwd2QAjbb3H6rN
4RCMnkH2rcOeyceIKZTA8x
7LIBRkpChxGlujYwa6YCLH
0aFnNJOkGAKKj7qrGREU0h
4Ab6XciUxDX

KeyError: 'tracks'